In [1]:
!pip install git+https://github.com/SheffieldSolar/PV_Live-API

  Cloning https://github.com/SheffieldSolar/PV_Live-API to /tmp/pip-req-build-2j3r5xqm
  Stored in directory: /tmp/pip-ephem-wheel-cache-i9pmt954/wheels/46/5c/ae/b7f2533e8d907b10cfcf5bf293178ed0e97251c0050576f2a7
Successfully built pvlive-api


In [4]:
import json
import requests
import pandas as pd
import numpy as np
from pprint import pprint
from datetime import datetime, date
#from pvlive_api import PVLive

In [5]:
df = pd.read_csv('data/consumption2017-P2.csv')
consumption_df = df[list(map(lambda x: len(x)>4,df.POSTCODE))]

In [ ]:
all_postcodes = []
for i in range(500):
    consumption_postcodes = list(consumption_df.iloc[100*i:100*i+100]['POSTCODE'])
    assert consumption_postcodes is not None
    payload = dict(postcodes=consumption_postcodes)
    
    assert payload is not None
    result = requests.post('https://api.postcodes.io/postcodes', data=payload).json()
    assert result['status'] is 200
    results = result['result']
    postcodes = []
    for result in results:
        if result['result'] is None:
            print(result)
            continue
        postcodes.append({'POSTCODE':result['result']['postcode'],'Longitude':result['result']['longitude'],'Latitude':result['result']['latitude']})
    
    all_postcodes.append(list(postcodes))

{'query': 'LA6 3GQ', 'result': None}
{'query': 'LA9 4', 'result': None}
{'query': 'LE1 1LF', 'result': None}
{'query': 'LE1 5NE', 'result': None}
{'query': 'LE1 7GP', 'result': None}


In [5]:
all_postcodes = sum(all_postcodes,[])

In [6]:
postcodes_df = pd.DataFrame(all_postcodes)
postcodes_df

,Latitude,Longitude,POSTCODE
0,54.125875,-2.778352,LA5 9UR
1,54.125347,-2.781985,LA5 9UT
2,54.135653,-2.771236,LA5 9UU
3,54.126280,-2.779402,LA5 9UW
4,54.135232,-2.771029,LA5 9UX
5,54.129656,-2.758836,LA5 9XA
6,54.130827,-2.756975,LA5 9XB
7,54.128824,-2.761086,LA5 9XE
8,54.129109,-2.760066,LA5 9XF
9,54.130839,-2.758001,LA5 9XG


In [40]:
postcodes_df = df.set_index('POSTCODE').join(postcodes_df.set_index('POSTCODE')).dropna()
postcodes_df

KeyError: 'POSTCODE'

In [26]:
long_to_X = (np.array(postcodes_df.Longitude)+8)//(10/90)
lat_to_Y = (np.array(postcodes_df.Latitude)-49)//(10/72)
tiles_data = list(zip(long_to_X,lat_to_Y))
tiles_df = pd.DataFrame(tiles_data, columns=['X','Y'], index=postcodes_df.index, dtype='int64')

def 

In [27]:
x_min = 990
y_min = 660

In [28]:
tiles_df = postcodes_df.join(tiles_df)

In [29]:
tiles_df = tiles_df.dropna()
tiles_df = tiles_df.reset_index()

In [30]:
#tiles_df.to_json('data/consumption-P2.json')

In [31]:
pv = PVLive()
pv_data = pd.DataFrame(pv.between(datetime(2016, 1, 1, 0, 0),
                     datetime(2019, 1, 1, 0, 0)),columns=['pes_id','datetime','generation'

In [17]:
pv_data.head()

,pes_id,datetime,generation
0,0,2016-01-01T00:00:00Z,0.0
1,0,2016-01-01T00:30:00Z,0.0
2,0,2016-01-01T01:00:00Z,0.0
3,0,2016-01-01T01:30:00Z,0.0
4,0,2016-01-01T02:00:00Z,0.0


In [32]:
max(tiles_df.Y)

60

In [48]:
tiles_df = pd.read_json('data/consumption-P1.json')

In [49]:
tiles_df.tail()

,POSTCODE,Number of meters,Consumption (kWh),Mean consumption (kWh),Median consumption (kWh),Latitude,Longitude,X,Y
99995,BN3 1EB,13,37534,2887,2280,50.825389,-0.158406,35,6
99996,BN3 1ED,39,102356,2625,2238,50.825306,-0.159361,35,6
99997,BN3 1EE,54,203331,3765,2633,50.824638,-0.159827,35,6
99998,BN3 1EF,54,132793,2459,2288,50.823996,-0.160136,35,6
99999,BN3 1EG,39,127016,3257,2142,50.823718,-0.158287,35,6
